<a href="https://colab.research.google.com/github/google/business_intelligence_group/blob/development/solutions/budget-adjustment/Naive_Budget_Adjustment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Budget Adjustment
#### Linear Programming
xxx
xxx

---

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [194]:
# @title Step.1 Create Input Widgets
!pip install pulp -q
import altair as alt
import pulp
import random
import ipywidgets
import pandas as pd
from IPython.display import display

def apply_style(text):
  span_style = (
      "<span style='font-size:15px; background: linear-gradient(transparent"
      " 90%, #4285F4 0%);'>"
      + text
      + '</style>'
  )
  return span_style

def make_widgets(n):
  widget_list = []
  set_style = {"description_width": "initial"}
  set_layout = ipywidgets.Layout(width='200px')
  for i in range(n):
    widget_list.append(
      ipywidgets.IntText(
        description = 'media' + str(i),
        value = random.randint(300, 1000) * 100,
        style = set_style,
        layout = set_layout,
      )
    )
    widget_list.append(
      ipywidgets.IntText(
        value = random.randint(500, 1000),
        style = set_style,
        layout = set_layout,
        )
    )
    widget_list.append(
      ipywidgets.FloatRangeSlider(
        value = [0.8, 1.2],
        min = 0,
        max = 2.0,
        step = 0.1,
        style = set_style,
        )
    )
  return widget_list

Number_of_Media = 10 #@param {type:"number"}
text_investment = ipywidgets.HTML(value=apply_style('Investment'))
text_conversions = ipywidgets.HTML(value=apply_style('Conversions'))
text_change_rate = ipywidgets.HTML(value=apply_style('Budget Change %'))
display(ipywidgets.GridBox([text_investment, text_conversions, text_change_rate],layout=ipywidgets.Layout(
    grid_template_columns="repeat(3, 300px)")))

widget_list = make_widgets(Number_of_Media)
ipywidgets.GridBox(widget_list, layout=ipywidgets.Layout(
    grid_template_columns="repeat(3, 300px)"))

GridBox(children=(HTML(value="<span style='font-size:15px; background: linear-gradient(transparent 90%, #4285F…

GridBox(children=(IntText(value=63400, description='media0', layout=Layout(width='200px'), style=DescriptionSt…

In [195]:
# @title Step.2 Do Linear Programming

# todo@rhirota override from sheet
values = []
for widget in widget_list:
  values.append(widget.value)
list_investment, list_conversions, list_change_rate = values[::3], values[1::3], values[2::3]

list_roi = [list_conversions[i] / list_investment[i] for i in range(len(list_investment))]
lift_lift = [1.1, 1.05, 1.03, 1.01]
# Linear Programming
for lift in lift_lift:
  problem = pulp.LpProblem('Problem Name', pulp.LpMaximize)

  # todo@rhirota LpVariable.dicts or something
  for i in range(Number_of_Media):
    exec('''media{} = pulp.LpVariable(
      "media{}",
      list_change_rate[i][0] * list_investment[i],
      list_change_rate[i][1] * list_investment[i]
      )'''.format(i, i))

  # Objective & Constraints
  text_objective = 'problem += media0'
  text_constraints_1 = 'problem += media0 * list_roi[0]'

  for i in range(1, Number_of_Media):
    text_objective += ' + media' + str(i)
    text_constraints_1 += ' + media' + str(i) + ' * list_roi[' + str(i) + ']'

  text_constraints_1 = text_constraints_1 + ' >= sum(list_conversions) * ' + str(lift)
  text_constraints_2 = text_objective + ' == sum(list_investment)'

  exec(text_objective)
  exec(text_constraints_1)
  exec(text_constraints_2)

  status = problem.solve()
  if pulp.LpStatus[status] == 'Optimal':
    print('An optimization that produces {:+,.0%} lift could be computed.'.format(lift-1))
    # print('='*20)
    # print(problem)
    # print('='*20)
    break
  else:
    print('Could not compute an optimization that would produce {:+,.0%} lift'.format(lift-1))
    print('='*20 + '\n')

Could not compute an optimization that would produce +10% lift

Could not compute an optimization that would produce +5% lift

An optimization that produces +3% lift could be computed.


/usr/local/lib/python3.9/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [247]:
# @title Step.3
list_optimal = []
list_media = []
for v in problem.variables():
  list_media.append(v.name)
  list_optimal.append(v.varValue)

df = pd.DataFrame(
    data = list(zip(list_investment, list_optimal, list_roi, list_conversions)),
    columns = ['investment', 'optimal_investment', 'roi', 'Original_CV'],
    index = list_media
    )
df['diff_investment'] = df['optimal_investment'] - df['investment']
df['Optimal_CV'] = df['optimal_investment'] * df['roi']
df['diff_conversions'] = df['Optimal_CV'] - df['Original_CV']

df.loc['total'] = df.sum(axis=0)
display(df)
print("\n" + "="*100 + "\n")
chart_invest = alt.Chart(df.reset_index()).transform_fold(
    ['investment', 'optimal_investment'],
).mark_line().mark_bar().encode(
    x=alt.X('key:O', title='', axis=alt.Axis(labels=False)),
    y=alt.Y('value:Q', title = ''),
    color=alt.Color('key:N', legend=None,),
    column=alt.Column('index:N', title='')
).properties(width=50, height=150,title='Investment')

chart_conversion = alt.Chart(df.reset_index()).transform_fold(
    ['Original_CV', 'Optimal_CV'],
).mark_line().mark_bar().encode(
    x=alt.X('key:O', title='',axis=alt.Axis(labelAngle=-30),sort=['Original', 'Optimal']),
    y=alt.Y('value:Q', title = ''),
    color=alt.Color('key:N', legend=None,),
    column=alt.Column('index:N', title='')
).properties(width=50, height=150, title='Conversions')

alt.vconcat(chart_invest, chart_conversion)

,investment,optimal_investment,roi,Original_CV,diff_investment,Optimal_CV,diff_conversions
media0,63400.0,76080.00,0.009353,593.0,12680.00,711.600000,118.600000
media1,76800.0,73935.93,0.006576,505.0,-2864.07,486.167248,-18.832752
media2,85800.0,68640.00,0.006970,598.0,-17160.00,478.400000,-119.600000
media3,98000.0,117600.00,0.008224,806.0,19600.00,967.200000,161.200000
media4,91000.0,72800.00,0.005593,509.0,-18200.00,407.200000,-101.800000
media5,98500.0,78800.00,0.007624,751.0,-19700.00,600.800000,-150.200000
media6,67300.0,80760.00,0.009688,652.0,13460.00,782.400000,130.400000
media7,75000.0,90000.00,0.011040,828.0,15000.00,993.600000,165.600000
media8,63500.0,75464.07,0.010079,640.0,11964.07,760.582753,120.582753
media9,73900.0,59120.00,0.007618,563.0,-14780.00,450.400000,-112.600000


alt.VConcatChart(...)